In [1]:
!pip install torch transformers datasets numpy pandas accelerate evaluate

In [3]:
import torch
import evaluate
import numpy as np

/Users/venkatakshaychintalapati/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/venkatakshaychintalapati/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/Users/venkatakshaychintalapati/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
device

'mps'

In [5]:
from datasets import load_dataset

dataset = load_dataset('yelp_review_full')

Found cached dataset parquet (/Users/venkatakshaychintalapati/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
help(load_dataset)

Help on function load_dataset in module datasets.load:

load_dataset(path: str, name: Optional[str] = None, data_dir: Optional[str] = None, data_files: Union[str, Sequence[str], Mapping[str, Union[str, Sequence[str]]], NoneType] = None, split: Union[str, datasets.splits.Split, NoneType] = None, cache_dir: Optional[str] = None, features: Optional[datasets.features.features.Features] = None, download_config: Optional[datasets.download.download_config.DownloadConfig] = None, download_mode: Union[datasets.download.download_manager.DownloadMode, str, NoneType] = None, verification_mode: Union[datasets.utils.info_utils.VerificationMode, str, NoneType] = None, ignore_verifications='deprecated', keep_in_memory: Optional[bool] = None, save_infos: bool = False, revision: Union[str, datasets.utils.version.Version, NoneType] = None, use_auth_token: Union[bool, str, NoneType] = None, task: Union[str, datasets.tasks.base.TaskTemplate, NoneType] = None, streaming: bool = False, num_proc: Optional[int

In [7]:
dataset['train'][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [7]:
help(AutoTokenizer)

Help on class AutoTokenizer in module transformers.models.auto.tokenization_auto:

class AutoTokenizer(builtins.object)
 |  This is a generic tokenizer class that will be instantiated as one of the tokenizer classes of the library when
 |  created with the [`AutoTokenizer.from_pretrained`] class method.
 |  
 |  This class cannot be instantiated directly using `__init__()` (throws an error).
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  register(config_class, slow_tokenizer_class=None, fast_tokenizer_class=None)
 |      Register a new tokenizer in this mapping.
 |      
 |      
 |      Args:
 |          config_class ([`PretrainedConfig`]):
 |              The configuration corresponding to the model to register.
 |          slow_tokenizer_class ([`PretrainedTokenizer`], *optional*):
 |              The slow tokenizer to register.
 |          slow_tokenizer_class ([`PretrainedTokenizerFast`], *opti

In [9]:
def tokenizing_function(examples):
    return tokenizer(examples['text'], padding = 'max_length', truncation=True)

In [10]:
tokenized_datasets = dataset.map(tokenizing_function,batched=True)

Loading cached processed dataset at /Users/venkatakshaychintalapati/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-8d2e6b489ecb59f2.arrow
Loading cached processed dataset at /Users/venkatakshaychintalapati/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-dccfbf93c4b41548.arrow


In [11]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))

small_eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /Users/venkatakshaychintalapati/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-19fbfc5363c6e196.arrow
Loading cached shuffled indices for dataset at /Users/venkatakshaychintalapati/.cache/huggingface/datasets/parquet/yelp_review_full-9c7006f5a2e02666/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ab88d0e54dfe7043.arrow


In [11]:
small_eval_dataset,small_eval_dataset['label']

(Dataset({
     features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 1000
 }),
 [2,
  4,
  1,
  4,
  3,
  4,
  2,
  3,
  2,
  3,
  0,
  0,
  3,
  2,
  2,
  1,
  3,
  1,
  2,
  2,
  1,
  2,
  3,
  1,
  1,
  3,
  4,
  0,
  0,
  2,
  2,
  2,
  1,
  3,
  4,
  0,
  0,
  1,
  3,
  2,
  0,
  2,
  0,
  0,
  3,
  0,
  3,
  2,
  3,
  0,
  1,
  1,
  3,
  3,
  4,
  4,
  1,
  4,
  1,
  3,
  1,
  0,
  0,
  1,
  4,
  1,
  4,
  3,
  2,
  4,
  1,
  0,
  3,
  3,
  4,
  1,
  2,
  1,
  0,
  4,
  4,
  4,
  2,
  3,
  3,
  1,
  4,
  0,
  4,
  2,
  3,
  0,
  0,
  0,
  3,
  4,
  0,
  0,
  1,
  4,
  4,
  0,
  0,
  1,
  1,
  0,
  4,
  2,
  2,
  1,
  1,
  4,
  0,
  4,
  0,
  3,
  2,
  0,
  4,
  4,
  4,
  2,
  0,
  0,
  0,
  1,
  3,
  0,
  2,
  0,
  3,
  2,
  2,
  2,
  0,
  3,
  4,
  3,
  0,
  1,
  0,
  1,
  0,
  0,
  4,
  3,
  3,
  1,
  3,
  0,
  3,
  4,
  0,
  2,
  1,
  3,
  1,
  3,
  1,
  3,
  1,
  0,
  2,
  0,
  1,
  1,
  0,
  0,
  3,
  3,
  3,
  3,
  3,
  1,
  1,
  0,
 

From above, we see that there are 5 labels. So that's what `num_labels` below will be

In [12]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased',num_labels=5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from transformers import TrainingArguments

In [54]:
help(TrainingArguments)

Help on class TrainingArguments in module transformers.training_args:

class TrainingArguments(builtins.object)
 |  TrainingArguments(output_dir: str, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, evaluation_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', warmup_ratio: float = 0.0, wa

`pip install accelerate` before you run the previous cell. You may also have to likely restart the notebook.

In [14]:
training_args = TrainingArguments(output_dir='test_trainer')

In [15]:
from transformers import Trainer

In [21]:
help(Trainer)

Help on class Trainer in module transformers.trainer:

class Trainer(builtins.object)
 |  Trainer(model: Union[transformers.modeling_utils.PreTrainedModel, torch.nn.modules.module.Module] = None, args: transformers.training_args.TrainingArguments = None, data_collator: Optional[transformers.data.data_collator.DataCollator] = None, train_dataset: Optional[torch.utils.data.dataset.Dataset] = None, eval_dataset: Union[torch.utils.data.dataset.Dataset, Dict[str, torch.utils.data.dataset.Dataset], NoneType] = None, tokenizer: Optional[transformers.tokenization_utils_base.PreTrainedTokenizerBase] = None, model_init: Optional[Callable[[], transformers.modeling_utils.PreTrainedModel]] = None, compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], Dict]] = None, callbacks: Optional[List[transformers.trainer_callback.TrainerCallback]] = None, optimizers: Tuple[torch.optim.optimizer.Optimizer, torch.optim.lr_scheduler.LambdaLR] = (None, None), preprocess_logits_for_metric

The metric we want to use for evaluation is Accuracy.

In [16]:
metric = evaluate.load('accuracy')

We need a `compute_metrics` function because the model predictions returned are logits, and we need to convert \
the logits back into predictions, so that we can compare them against our target labels to compute accuracy.

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

The `evaluation_strategy` parameter in TrainingArguments is used to monitor eval metrics at the end of every epoch.

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir = 'test_trainer', evaluation_strategy ='epoch')

### Trainer

Time to bring in the big guns. `Trainer` class is a feature-complete training and eval loop built by transformers for PyTorch. Can be \
used not only with models on Transformers but your own pre-trained models built using `torch.nn.Module`

In [26]:
help(Trainer)

Help on class Trainer in module transformers.trainer:

class Trainer(builtins.object)
 |  Trainer(model: Union[transformers.modeling_utils.PreTrainedModel, torch.nn.modules.module.Module] = None, args: transformers.training_args.TrainingArguments = None, data_collator: Optional[transformers.data.data_collator.DataCollator] = None, train_dataset: Optional[torch.utils.data.dataset.Dataset] = None, eval_dataset: Union[torch.utils.data.dataset.Dataset, Dict[str, torch.utils.data.dataset.Dataset], NoneType] = None, tokenizer: Optional[transformers.tokenization_utils_base.PreTrainedTokenizerBase] = None, model_init: Optional[Callable[[], transformers.modeling_utils.PreTrainedModel]] = None, compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], Dict]] = None, callbacks: Optional[List[transformers.trainer_callback.TrainerCallback]] = None, optimizers: Tuple[torch.optim.optimizer.Optimizer, torch.optim.lr_scheduler.LambdaLR] = (None, None), preprocess_logits_for_metric

In [19]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [20]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_always_pu

In [25]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics = compute_metrics
)

In [26]:
trainer.train().to(device)

  0%|          | 0/375 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.1344172954559326, 'eval_accuracy': 0.555, 'eval_runtime': 43.5849, 'eval_samples_per_second': 22.944, 'eval_steps_per_second': 2.868, 'epoch': 1.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.0174952745437622, 'eval_accuracy': 0.568, 'eval_runtime': 42.6076, 'eval_samples_per_second': 23.47, 'eval_steps_per_second': 2.934, 'epoch': 2.0}


  0%|          | 0/125 [00:00<?, ?it/s]

{'eval_loss': 1.0102664232254028, 'eval_accuracy': 0.596, 'eval_runtime': 42.5238, 'eval_samples_per_second': 23.516, 'eval_steps_per_second': 2.94, 'epoch': 3.0}
{'train_runtime': 587.05, 'train_samples_per_second': 5.11, 'train_steps_per_second': 0.639, 'train_loss': 1.0624818522135417, 'epoch': 3.0}


AttributeError: 'TrainOutput' object has no attribute 'to'

The significance of the `[CLS]` token that is added at the start of every sequence input is that, the model encodes that entire sequence's context into this single CLS token.